In [1]:
# 화면 비율 넓게 설정
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max.colwidth',150)

C:\Users\정승우\AppData\Local\Temp\ipykernel_3468\1083280195.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def crawler(category_num):
    from bs4 import BeautifulSoup
    import urllib.request
    import re

    #웹브라우져 헤더 추가
    #User-Agent를 조작하는 경우
    hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    req = urllib.request.Request('https://prod.danawa.com/list/?cate='+category_num, headers = hdr)
    data = urllib.request.urlopen(req).read()
    page = data.decode('utf-8','ignore')
    soup = BeautifulSoup(page, 'html.parser')
    prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
    return prod_items

In [3]:
import re
def cpu_crawler(link):
    prod_items = crawler(link)
    i=0
    partslist = []
    for item in prod_items:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box--full').get_text()
        details = re.split("[/'\n']", details) # 상세정보들
        if link[-6:] == '113973': # intel 소캣 split
            socket = details[0].strip()[5:-1] if details else 'N/A' 
        elif link[-6:] == '113990': # amd 소켓 splite
            socket = details[0].strip()[6:-1] if details else 'N/A'
        integrated_graphics = True if '내장그래픽: 탑재' in details else False # 내장그래픽
        for detail in details:
            if '메모리 규격' in detail:
                memory_type = str(detail[8:])
            if '시네벤치R23(싱글)' in detail:
                singlebench = detail[13:]
            if '시네벤치R23(멀티)' in detail:
                multibench = detail[13:]
        print([name, price, socket, integrated_graphics, memory_type, singlebench, multibench])
        partslist.append([name, price, socket, integrated_graphics, memory_type, singlebench, multibench])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Socket', 'Integrated Graphics','메모리 타입','시네벤치R23(싱글)','시네벤치R23(멀티)'])
    return result

intel_link ='113973'
amd_link = '113990'
intel_cpu = cpu_crawler(intel_link)
amd_cpu = cpu_crawler(amd_link)

['인텔 코어i5-14세대 14400F (랩터레이크 리프레시)', '283,080', '', False, 'DDR5, DDR4', '1790', '17804']
['인텔 코어i7-14세대 14700K (랩터레이크 리프레시)', '583,640', '', True, 'DDR5, DDR4', '2069', '34818']
['인텔 코어i5-12세대 12400F (엘더레이크)', '152,030', '', False, 'DDR5, DDR4', '1605', '12201']
['인텔 코어i5-14세대 14500 (랩터레이크 리프레시)', '347,270', '', True, 'DDR5, DDR4', '1934', '22045']
['인텔 코어i9-14세대 14900K (랩터레이크 리프레시)', '814,470', '', True, 'DDR5, DDR4', '2277', '40146']
['인텔 코어i5-13세대 13400F (랩터레이크)', '238,150', '', False, 'DDR5, DDR4', '1789', '14621']
['인텔 코어i7-14세대 14700F (랩터레이크 리프레시)', '516,300', '', False, 'DDR5, DDR4', '2091', '32090']
['인텔 코어i7-14세대 14700KF (랩터레이크 리프레시)', '547,920', '', False, 'DDR5, DDR4', '2145', '35218']
['인텔 코어i7-12세대 12700K (엘더레이크)', '404,480', '', True, 'DDR5, DDR4', '1960', '22788']
['인텔 코어i3-12세대 12100F (엘더레이크)', '104,850', '', False, 'DDR5, DDR4', '1666', '8376']
['인텔 코어i7-13세대 13700KF (랩터레이크)', '462,500', '', False, 'DDR5, DDR4', '1978', '30354']
['인텔 코어i5-14세대 14400 (랩터레이크 리프레시)', '32

In [4]:
amd_cpu.tail()

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
25,AMD 라이젠9-5세대 7900X (라파엘),"532,230",,True,DDR5,2034,29349
26,AMD 라이젠9-4세대 5900X (버미어),"457,450",,False,DDR4,1578,20811
27,AMD 라이젠7-4세대 5700G (세잔),"340,330",,True,DDR4,1507,13898
28,AMD 라이젠9-4세대 5950X (버미어),"656,680",,False,DDR4,1507,13898
29,AMD 라이젠7-5세대 7700X (라파엘),"510,000",,True,DDR5,2018,20059


In [5]:
intel_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,인텔 코어i5-14세대 14400F (랩터레이크 리프레시),"283,080",,False,"DDR5, DDR4",1790,17804
1,인텔 코어i7-14세대 14700K (랩터레이크 리프레시),"583,640",,True,"DDR5, DDR4",2069,34818
2,인텔 코어i5-12세대 12400F (엘더레이크),"152,030",,False,"DDR5, DDR4",1605,12201
3,인텔 코어i5-14세대 14500 (랩터레이크 리프레시),"347,270",,True,"DDR5, DDR4",1934,22045
4,인텔 코어i9-14세대 14900K (랩터레이크 리프레시),"814,470",,True,"DDR5, DDR4",2277,40146
5,인텔 코어i5-13세대 13400F (랩터레이크),"238,150",,False,"DDR5, DDR4",1789,14621
6,인텔 코어i7-14세대 14700F (랩터레이크 리프레시),"516,300",,False,"DDR5, DDR4",2091,32090
7,인텔 코어i7-14세대 14700KF (랩터레이크 리프레시),"547,920",,False,"DDR5, DDR4",2145,35218
8,인텔 코어i7-12세대 12700K (엘더레이크),"404,480",,True,"DDR5, DDR4",1960,22788
9,인텔 코어i3-12세대 12100F (엘더레이크),"104,850",,False,"DDR5, DDR4",1666,8376


In [6]:
intel_cpu.to_csv('intel.csv',encoding='cp949', index=False)
intel_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,인텔 코어i5-14세대 14400F (랩터레이크 리프레시),"283,080",,False,"DDR5, DDR4",1790,17804
1,인텔 코어i7-14세대 14700K (랩터레이크 리프레시),"583,640",,True,"DDR5, DDR4",2069,34818
2,인텔 코어i5-12세대 12400F (엘더레이크),"152,030",,False,"DDR5, DDR4",1605,12201
3,인텔 코어i5-14세대 14500 (랩터레이크 리프레시),"347,270",,True,"DDR5, DDR4",1934,22045
4,인텔 코어i9-14세대 14900K (랩터레이크 리프레시),"814,470",,True,"DDR5, DDR4",2277,40146
5,인텔 코어i5-13세대 13400F (랩터레이크),"238,150",,False,"DDR5, DDR4",1789,14621
6,인텔 코어i7-14세대 14700F (랩터레이크 리프레시),"516,300",,False,"DDR5, DDR4",2091,32090
7,인텔 코어i7-14세대 14700KF (랩터레이크 리프레시),"547,920",,False,"DDR5, DDR4",2145,35218
8,인텔 코어i7-12세대 12700K (엘더레이크),"404,480",,True,"DDR5, DDR4",1960,22788
9,인텔 코어i3-12세대 12100F (엘더레이크),"104,850",,False,"DDR5, DDR4",1666,8376


In [7]:
amd_cpu.to_csv('amd.csv',encoding='cp949', index=False)
amd_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,AMD 라이젠5-5세대 7500F (라파엘),"192,240",,False,DDR5,1824,13824
1,AMD 라이젠7-5세대 7800X3D (라파엘),"526,980",,True,DDR5,1788,18208
2,AMD 라이젠5-5세대 8600G (피닉스),"262,380",,True,DDR5,1662,13513
3,AMD 라이젠5-4세대 5600 (버미어),"151,900",,False,DDR4,1460,10906
4,AMD 라이젠5 PRO 4650G (르누아르),"101,270",,True,DDR4,1460,10906
5,AMD 라이젠5-4세대 5600G (세잔),"280,000",,True,DDR4,1446,10489
6,AMD 라이젠5-5세대 7600 (라파엘),"253,430",,True,DDR5,1864,14387
7,AMD 라이젠7-4세대 5700X3D (버미어),"285,390",,False,DDR4,1864,14387
8,AMD 라이젠9-5세대 7950X3D (라파엘),"832,360",,True,DDR5,2060,36372
9,AMD 라이젠9-5세대 7900 (라파엘),"514,950",,True,DDR5,1969,24527


In [8]:
# 램 가격 크롤링
ddr4_link = '1131326'
ddr5_link = '11341201'

def memory_crawling(link):
    crolled_data = crawler(link)
    # 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
    i=0
    partslist = list()
    for item in crolled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        speed = details[2]
        partslist.append([name, price, speed])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price','Speed'])
    return result
ddr4 = memory_crawling(ddr4_link)
ddr5 = memory_crawling(ddr5_link)
ddr4

,Product Name,Price,Speed
0,ESSENCORE KLEVV DDR4-3200 CL22 파인인포,"73,310",3200MHz (PC4-25600)
1,삼성전자 DDR4-3200,"92,230",3200MHz (PC4-25600)
2,삼성전자 DDR4-2666,"43,090",2666MHz (PC4-21300)
3,ESSENCORE KLEVV DDR4-3600 CL18 BOLT XR 패키지 서린,"98,990",3600MHz (PC4-28800)
4,G.SKILL DDR4-3600 CL18 TRIDENT Z RGB 패키지,"222,220",3600MHz (PC4-28800)
5,TeamGroup T-Force DDR4-3600 CL18 Delta RGB 화이트 패키지 서린,"240,000",3600MHz (PC4-28800)
6,마이크론 Crucial DDR4-3200 CL22 대원씨티에스,"42,060",3200MHz (PC4-25600)
7,마이크론 Crucial DDR4-3200 CL22,"82,440",3200MHz (PC4-25600)
8,ESSENCORE KLEVV DDR4-3200 CL16 BOLT X 서린,"44,890",3200MHz (PC4-25600)
9,G.SKILL DDR4-3600 CL16 TRIDENT Z NEO C 패키지,"141,400",3600MHz (PC4-28800)


In [9]:
ddr5

,Product Name,Price,Speed
0,TeamGroup DDR5-5600 CL46 Elite 서린,"124,000",5600MHz (PC5-44800)
1,삼성전자 DDR5-5600,"133,970",5600MHz (PC5-44800)
2,마이크론 Crucial DDR5-5600 CL46 PRO 패키지 대원씨티에스,"370,830",5600MHz (PC5-44800)
3,TeamGroup T-CREATE DDR5-5600 CL46 CLASSIC 패키지 서린,"290,000",5600MHz (PC5-44800)
4,마이크론 Crucial DDR5-5600 CL46 대원씨티에스,"120,700",5600MHz (PC5-44800)
5,ESSENCORE KLEVV DDR5-6000 CL30 CRAS V RGB WHITE 패키지 서린,"350,000",6000MHz (PC5-48000)
6,ESSENCORE KLEVV DDR5-5600 CL46 파인인포,"117,480",5600MHz (PC5-44800)
7,G.SKILL DDR5-6000 CL30 RIPJAWS M5 RGB 화이트 패키지,"338,000",6000MHz (PC5-48000)
8,SK하이닉스 DDR5-5600,"168,060",5600MHz (PC5-44800)
9,TeamGroup T-Force DDR5-6000 CL38 Delta RGB 화이트 패키지 서린,"340,000",6000MHz (PC5-48000)


In [10]:
ddr4.to_csv('ddr4.csv',encoding='cp949', index=False)
ddr5.to_csv('ddr5.csv',encoding='cp949', index=False)

In [11]:
# 메인보드 크롤링
def board_crawling(link):
    crolled_data = crawler(link)
    i=0
    partslist = list()
    for item in crolled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        # print(i,details)
        for idx in range(len(details)):
            if '메모리' in details[idx]:
                memory_type = str(details[idx][4:])
                memory_speed = str(details[idx+1].split()[0])
                break
        size = details[2].split()[0]
        partslist.append([name, price, memory_type, memory_speed, size])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Memory_Type','Memory_Speed', 'Size'])
    return result
amd_board_link = '1131249'
intel_board_link = '11345282'
amd_board = board_crawling(amd_board_link)
amd_board

,Product Name,Price,Memory_Type,Memory_Speed,Size
0,ASRock B650M PG Lightning 에즈윈,"160,000",DDR5,7200MHz,M-ATX
1,MSI MAG B650M 박격포 WIFI,"249,000",DDR5,7600MHz,M-ATX
2,ASUS PRIME A520M-A II 대원씨티에스,"81,930",DDR4,4866MHz,M-ATX
3,MSI MAG B650 토마호크 WIFI,"290,870",DDR5,7600MHz,ATX
4,ASUS TUF Gaming B550M-PLUS STCOM,"155,420",DDR4,4800MHz,M-ATX
5,ASRock B650M Pro RS 대원씨티에스,"179,820",DDR5,7200MHz,M-ATX
6,GIGABYTE B650M K 피씨디렉트,"159,500",DDR5,8000MHz,M-ATX
7,GIGABYTE B650M K 제이씨현,"168,990",DDR5,8000MHz,M-ATX
8,ASRock B450M-HDV R4.0 대원씨티에스,"66,500",DDR4,3200MHz,M-ATX
9,ASRock B650M PG Lightning 대원씨티에스,"149,740",DDR5,7200MHz,M-ATX


In [12]:
intel_board = board_crawling(intel_board_link)
intel_board

,Product Name,Price,Memory_Type,Memory_Speed,Size
0,MSI MAG B760M 박격포 맥스 WIFI,"213,990",DDR5,7000MHz,M-ATX
1,ASUS PRIME H610M-K D4 인텍앤컴퍼니,"97,980",DDR4,3200MHz,M-ATX
2,GIGABYTE B760M AORUS ELITE 피씨디렉트,"188,680",DDR5,7800MHz,M-ATX
3,ASUS TUF Gaming B760M-PLUS II 코잇,"198,000",DDR5,7800MHz,M-ATX
4,MSI PRO B760M-A DDR4 II,"149,000",DDR4,5333MHz,M-ATX
5,GIGABYTE B760M AORUS ELITE D4 제이씨현,"176,900",DDR4,5333MHz,M-ATX
6,GIGABYTE Z790 AORUS ELITE 피씨디렉트,"337,360",DDR5,7600MHz,ATX
7,MSI MAG B760M 박격포 II,"195,000",DDR5,7800MHz,M-ATX
8,GIGABYTE B760M DS3H D4 피씨디렉트,"137,990",DDR4,5333MHz,M-ATX
9,ASUS ROG STRIX B760-G GAMING WIFI 코잇,"231,480",DDR5,7800MHz,M-ATX


In [13]:
amd_board.to_csv('amd_board.csv',encoding='cp949', index=False)
intel_board.to_csv('intel_board.csv',encoding='cp949', index=False)

In [14]:
# GPU 크롤링
def gpu_crawling(link):
    crawled_data = crawler(link)
    i=0
    partslist = list()
    for item in crawled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        for idx in range(len(details)):
            if '정격파워' in details[idx]:
                # power_usage = str(details[idx].split()[1])
                power_type = str(details[idx].split()[1])
            if '가로(길이)' in details[idx]:
                size = str(details[idx].split()[1])
                break
        partslist.append([name, price, power_type, size])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Power_Type', '가로(길이)'])
    return result
gpu_link = '112753'
gpu = gpu_crawling(gpu_link)
gpu

,Product Name,Price,Power_Type,가로(길이)
0,이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB,"417,790",600W,249.9mm
1,MSI 지포스 RTX 4070 SUPER 게이밍 X 슬림 D6X 12GB 트라이프로져3,"976,990",650W,307mm
2,이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB,"557,190",650W,249.9mm
3,ZOTAC GAMING 지포스 RTX 4060 TWIN Edge OC D6 8GB,"412,760",500W,225.3mm
4,MSI 지포스 RTX 4060 벤투스 2X 블랙 OC D6 8GB,"426,880",550W,199mm
5,MSI 지포스 RTX 4070 Ti SUPER 벤투스 3X OC D6X 16GB,"1,159,780",700W,308mm
6,MSI 지포스 RTX 4070 Ti SUPER 게이밍 X 슬림 D6X 16GB 트라이프로져3,"1,243,730",700W,307mm
7,갤럭시 GALAX 지포스 RTX 4060 WHITE OC D6 8GB,"431,790",550W,237mm
8,MSI 지포스 RTX 4080 SUPER 게이밍 X 슬림 D6X 16GB 트라이프로져3,"1,658,470",850W,322mm
9,XFX 라데온 RX 6800 SWFT 319 CORE D6 16GB,"500,330",650W,340mm


In [15]:
gpu.to_csv('gpu.csv',encoding='cp949', index=False)

In [20]:
!pip install selenium

In [43]:
# 파워 크롤링은 다른 방식으로 실시하였음
# why? 페이지 5까지 크롤링 가능해서 최대 150개까지 크롤링 가능하기 때문

import time
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

# 브라우저 띄우지 않고 하기
options = ChromeOptions()
options.add_argument('headless')

url = 'https://prod.danawa.com/list/?cate=112777'
driver = Chrome()
driver.get(url)

time.sleep(2)

j=0
partslist = list() # make empty list

for page in range(1,6):# 5page
    soup = BeautifulSoup(driver.page_source)
    product_li_tags = soup.select('li.prod_item.prod_layer')
    for li in product_li_tags:
        name = li.select_one('p.prod_name a').text.strip()
        price = li.select_one('p.price_sect a strong').text.strip()
        details = li.select_one('div.spec_list').get_text().split('/')
        for idx in range(len(details)):
            if '파워' in details[idx]:
                power_size = str(details[idx].split()[0])
            if '정격출력' in details[idx]:
                output = str(details[idx].split()[1])
                break
        img_link = li.select_one('div.thumb_image img').get('data-original')
        if img_link == None:
            img_link = li.select_one('div.thumb_image img').get('src')[2:] # img link가 //으로 들어와서 //없에버림
        # print(j,name, price, power_size, output, img_link)
        partslist.append([name, price, power_size, output, img_link])
        j+=1
power = pd.DataFrame(partslist, columns=['Product Name', 'Price', '파워규격', '정격출력','img_link'])
power.to_csv('power.csv',encoding='cp949', index=False)
power

,Product Name,Price,파워규격,정격출력,img_link
0,마이크로닉스 Classic II 풀체인지 700W 80PLUS브론즈 ATX3.1,"78,031",ATX,700W,img.danawa.com/prod_img/500000/700/642/img/49642700_1.jpg?shrink=130:130&_v=20240703124116
1,앱코 SETTLER 하이브리드 PCIE5.1 STH-700B ETA BRONZE 화이트,"65,000",ATX,700W,img.danawa.com/prod_img/500000/012/742/img/49742012_1.jpg?shrink=130:130&_v=20240726174519
2,마이크로닉스 Classic II 풀체인지 600W 80PLUS브론즈 ATX3.1,"62,500",ATX,600W,img.danawa.com/prod_img/500000/652/642/img/49642652_1.jpg?shrink=130:130&_v=20240603170345
3,잘만 GigaMax III 850W 80PLUS브론즈 모듈러 ATX3.1,"90,540",ATX,850W,img.danawa.com/prod_img/500000/807/615/img/29615807_1.jpg?shrink=130:130&_v=20240513112215
4,마이크로닉스 Classic II 풀체인지 500W 80PLUS브론즈 ATX3.1,"54,500",ATX,500W,//img.danawa.com/prod_img/500000/352/642/img/49642352_1.jpg?shrink=130:130&_v=20240603170331
...,...,...,...,...,...
150,AONE 시그니처 750W 80PLUS브론즈 풀모듈러 베이직 ATX3.1,"65,000",ATX,750W,//img.danawa.com/prod_img/500000/734/912/img/39912734_1.jpg?shrink=130:130&_v=20240710100340
151,쿨러마스터 MWE 700 BRONZE V2 230V,"71,970",ATX,700W,//img.danawa.com/prod_img/500000/805/400/img/12400805_1.jpg?shrink=130:130&_v=20210107133641
152,마이크로닉스 Compact SFX 700W 80PLUS골드,"139,000",M-ATX(SFX),700W,//img.danawa.com/prod_img/500000/575/759/img/10759575_1.jpg?shrink=130:130&_v=20240507153840
153,시소닉 NEW FOCUS GX-850 GOLD 풀모듈러 ATX3.0,"230,000",ATX,850W,//img.danawa.com/prod_img/500000/773/416/img/28416773_1.jpg?shrink=130:130&_v=20240528142620


In [16]:
# 케이스 크롤링 - 이전방식

def case_crawling(link):
    crawled_data = crawler(link)
    i=0
    partslist = list()
    for item in crawled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        # link = 
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        for idx in range(len(details)):
            if 'VGA 장착 길이' in details[idx]:
                vga_length = str(details[idx].split()[3])
            if 'CPU쿨러 장착높이' in details[idx]:
                cpuCooler_height = str(details[idx].split()[2])
                break
        partslist.append([name, price, vga_length, cpuCooler_height])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'VGA 장착길이', 'CPU쿨러 장착높이'])
    return result

case_link = '113971'
case = case_crawling(case_link)
case

,Product Name,Price,VGA 장착길이,CPU쿨러 장착높이
0,DAVEN D6 MESH 강화유리,"37,310",330mm,173mm
1,앱코 U30 마린,"37,900",400mm,165mm
2,darkFlash DS900 ARGB 강화유리,"52,690",425mm,170mm
3,BRAVOTEC GUARDIAN 3100M V2 타이탄 글래스,"87,730",380mm,180mm
4,마이크로닉스 COOLMAX 스테디,"39,140",355mm,165mm
5,앱코 NCORE G30 트루포스,"45,500",340mm,175mm
6,darkFlash DLX21 RGB MESH 강화유리,"92,740",400mm,180mm
7,DAVEN AQUARIUM,"55,950",415mm,163mm
8,앱코 U20M 큐빅 미니,"29,900",350mm,165mm
9,Antec C8 MESH,"126,500",440mm,175mm


In [17]:
# 파워 크롤링 - 이전방식

def power_crawling(link):
    crawled_data = crawler(link)
    i=0
    partslist = list()
    for item in crawled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        # link = 
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        for idx in range(len(details)):
            if '파워' in details[idx]:
                power_size = str(details[idx].split()[0])
            if '정격출력' in details[idx]:
                output = str(details[idx].split()[1])
                break
        partslist.append([name, price, power_size, output])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', '파워규격', '정격출력'])
    return result

power_link = '11338807' # 80 PLUS 인증 파워만 가져옴
power = power_crawling(power_link)
power.to_csv('power.csv',encoding='cp949', index=False)
power

,Product Name,Price,파워규격,정격출력
0,마이크로닉스 Classic II 풀체인지 700W 80PLUS브론즈 ATX3.1,"78,031",ATX,700W
1,마이크로닉스 Classic II 풀체인지 600W 80PLUS브론즈 ATX3.1,"62,500",ATX,600W
2,잘만 GigaMax III 850W 80PLUS브론즈 모듈러 ATX3.1,"90,540",ATX,850W
3,마이크로닉스 Classic II 풀체인지 500W 80PLUS브론즈 ATX3.1,"54,500",ATX,500W
4,마이크로닉스 Classic II 850W 80PLUS골드 풀모듈러 ATX3.1,"159,008",ATX,850W
5,마이크로닉스 Classic II 풀체인지 800W 80PLUS브론즈 ATX3.1,"92,915",ATX,800W
6,잘만 MegaMax 600W 80PLUS스탠다드,"50,650",ATX,600W
7,시소닉 VERTEX GX-1000 GOLD 풀모듈러 ATX3.0,"254,000",ATX,1000W
8,마이크로닉스 Classic II 풀체인지 700W 80PLUS브론즈 ATX3.1 화이트,"86,650",ATX,700W
9,마이크로닉스 Classic II 1050W 80PLUS골드 230V EU 풀모듈러 화이트,"185,000",ATX,1050W


In [19]:
# 다나와 인텔 CPU 가격만 크롤링 코드
from bs4 import BeautifulSoup
import urllib.request
import re

#User-Agent를 조작하는 경우(아이폰에서 사용하는 사파리 브라우져의 헤더)
hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
data ='https://prod.danawa.com/list/?cate=113973'
#웹브라우져 헤더 추가
req = urllib.request.Request(data, \
                            headers = hdr)

data = urllib.request.urlopen(req).read()
page = data.decode('utf-8','ignore')
soup = BeautifulSoup(page, 'html.parser')
crolled_data = soup.find_all(['p','strong'],attrs={'class':'price_sect'})

# 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
i=0
partslist = list()
for item in crolled_data:
    if i>=4 and i!=9:
        # strip 메서드로 앞 뒤 공백 제거
        print(item.select_one('strong').get_text())
    i+=1
print(partslist)


283,080
281,740
271,340
312,410
583,640
535,990
152,030
134,624
123,360
150,000
347,270
348,900
338,260
398,750
814,470
791,290
659,040
152,030
238,150
227,090
217,660
247,350
516,300
500,000
507,620
601,170
547,920
506,370
607,890
404,480
331,190
342,990
328,940
538,290
104,850
96,352
86,910
104,740
462,500
454,420
444,990
510,640
321,820
285,730
271,740
379,180
445,390
424,070
481,380
390,920
396,510
384,870
419,810
162,710
162,550
152,820
210,100
969,370
563,860
564,300
539,000
700,550
358,990
333,160
322,150
363,910
155,410
111,970
129,210
149,740
295,300
259,990
241,800
243,000
415,690
154,370
123,260
118,140
100,150
402,050
422,600
272,920
268,960
199,690
179,000
167,450
266,350
404,430
368,690
417,550
773,680
763,140
899,360
208,440
182,050
194,560
257,890
352,790
303,900
291,000
548,700
377,920
318,070
334,530
434,740
233,630
206,290
207,320
228,770
269,390
274,590
261,640
271,650
647093
[]
